In [3]:
import pandas as pd

Path = "./data/"

# Read interactions_test.csv
df_interactions_test = pd.read_csv(Path + 'interactions_test.csv')

# Read interactions_train.csv
df_interactions_train = pd.read_csv(Path + 'interactions_train.csv')

# Read RAW_interactions.csv
df_raw_interactions = pd.read_csv(Path + 'RAW_interactions.csv')

# Read RAW_recipes.csv
df_raw_recipes = pd.read_csv(Path + 'RAW_recipes.csv')


In [7]:
# List of DataFrames with names
dataframes = [
    ("df_interactions_test", df_interactions_test),
    ("df_interactions_train", df_interactions_train),
    ("df_raw_interactions", df_raw_interactions),
    ("df_raw_recipes", df_raw_recipes)
]

# Apply describe() and print shape for each DataFrame
for name, df in dataframes:
    print(f"DataFrame: {name}")
    print(f"Shape: {df.shape}")
    print(df.describe())
    print('\n')  # Add a newline for separation

DataFrame: df_interactions_test
Shape: (12455, 6)
            user_id      recipe_id        rating             u              i
count  1.245500e+04   12455.000000  12455.000000  12455.000000   12455.000000
mean   2.912689e+07  209323.124528      4.213087  12288.499318  115488.123485
std    2.334357e+08  135001.832923      1.338503   6897.751394   50448.663212
min    1.533000e+03     120.000000      0.000000      2.000000     102.000000
25%    1.698420e+05   94616.000000      4.000000   6428.500000   76904.000000
50%    3.829540e+05  195040.000000      5.000000  12023.000000  127793.000000
75%    8.016370e+05  314928.500000      5.000000  17985.500000  160024.000000
max    2.002255e+09  537716.000000      5.000000  25074.000000  178264.000000


DataFrame: df_interactions_train
Shape: (698901, 6)
            user_id      recipe_id         rating              u  \
count  6.989010e+05  698901.000000  698901.000000  698901.000000   
mean   1.247694e+07  156173.409849       4.574090    4249.

In [8]:
# Print first 5 rows for each DataFrame
for name, df in dataframes:
    print(f"DataFrame: {name}")
    print(f"Shape: {df.shape}")
    print(df.head(5))
    print('\n')  # Add a newline for separation

DataFrame: df_interactions_test
Shape: (12455, 6)
   user_id  recipe_id        date  rating   u       i
0     8937      44551  2005-12-23     4.0   2  173538
1    56680     126118  2006-10-07     4.0  16  177847
2   349752     219596  2008-04-12     0.0  26   89896
3   628951      82783  2007-11-13     2.0  45  172637
4    92816     435013  2013-07-31     3.0  52  177935


DataFrame: df_interactions_train
Shape: (698901, 6)
   user_id  recipe_id        date  rating      u       i
0     2046       4684  2000-02-25     5.0  22095   44367
1     2046        517  2000-02-25     5.0  22095   87844
2     1773       7435  2000-03-13     5.0  24732  138181
3     1773        278  2000-03-13     4.0  24732   93054
4     2046       3431  2000-04-07     5.0  22095  101723


DataFrame: df_raw_interactions
Shape: (1132367, 5)
   user_id  recipe_id        date  rating  \
0    38094      40893  2003-02-17       4   
1  1293707      40893  2011-12-21       5   
2     8937      44394  2002-12-01       4 

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from statistics import mean